# Trading Strategy ML - Google Colab Setup

This notebook sets up and runs the Multi-Factor Momentum Trading Strategy with ML Enhancement on Google Colab with GPU support.

## Features
- GPU-accelerated training
- Real-time data collection
- Advanced ML models (CNN+LSTM)
- Comprehensive backtesting
- Performance analysis


## 1. Setup and Installation


In [ ]:
# Check GPU availability
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))
print("CUDA available:", tf.test.is_built_with_cuda())

# Enable GPU memory growth
if tf.config.list_physical_devices('GPU'):
    try:
        for gpu in tf.config.list_physical_devices('GPU'):
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled")
    except RuntimeError as e:
        print(f"GPU memory growth error: {e}")


In [ ]:
# Install required packages
!pip install -q yfinance alpha-vantage pandas-datareader
!pip install -q TA-Lib
!pip install -q tensorflow-gpu
!pip install -q torch torchvision
!pip install -q scikit-learn xgboost optuna lightgbm
!pip install -q backtrader zipline-reloaded arch quantlib
!pip install -q plotly matplotlib seaborn streamlit
!pip install -q empyrical ffn pyfolio
!pip install -q python-dotenv requests tqdm joblib

print("All packages installed successfully!")


In [ ]:
# Clone the repository (replace with your GitHub URL)
!git clone https://github.com/CatalinMoldova/trading-strategy-ml.git

# Change to the project directory
%cd trading-strategy-ml

# Install project requirements
!pip install -r requirements_colab.txt

print("Repository cloned and requirements installed!")


## 2. Import Libraries and Setup


In [ ]:
# Import necessary libraries
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Add project src to path
sys.path.append('src')

# Import project modules
from data_pipeline.market_data_collector import MarketDataCollector
from data_pipeline.indicator_engine import IndicatorEngine
from data_pipeline.feature_engineer import FeatureEngineer
from ml_models.cnn_lstm_model import CNNLSTMModel
from ml_models.random_forest_model import RandomForestModel
from ml_models.ensemble_predictor import EnsemblePredictor
from strategy.signal_generator import SignalGenerator
from strategy.position_sizer import PositionSizer
from strategy.risk_manager import RiskManager
from backtesting.backtest_engine import BacktestEngine
from backtesting.performance_analyzer import PerformanceAnalyzer

print("All libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU devices: {tf.config.list_physical_devices('GPU')}")


## 3. Data Collection and Preparation


In [ ]:
# Initialize data collector
collector = MarketDataCollector()

# Define symbols to trade
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'META', 'NFLX']

# Collect historical data
print("Collecting historical data...")
data = {}
for symbol in symbols:
    try:
        df = collector.get_historical_data(symbol, period='2y', interval='1d')
        data[symbol] = df
        print(f"✓ {symbol}: {len(df)} records")
    except Exception as e:
        print(f"✗ {symbol}: Error - {e}")

print(f"\nData collection complete! Collected data for {len(data)} symbols.")


## 4. Model Training with GPU


In [ ]:
# Train CNN+LSTM model with GPU
print("Training CNN+LSTM model...")

# Initialize model
cnn_lstm = CNNLSTMModel(
    time_steps=60,
    n_features=20,  # Adjust based on your features
    learning_rate=0.001
)

# Build model
model = cnn_lstm.build_model()
print(f"Model built with {model.count_params()} parameters")

# Prepare training data (simplified example)
# In practice, you'd use your actual processed data
X_train = np.random.randn(1000, 60, 20)  # Example data
y_train = np.random.randn(1000, 1)        # Example targets

# Train model
history = cnn_lstm.train(
    X_train, y_train,
    epochs=10,  # Reduced for demo
    batch_size=32,
    validation_split=0.2
)

print("CNN+LSTM training complete!")

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()


## 5. Save Your Work


In [ ]:
# Save models
cnn_lstm.save_model('cnn_lstm_model.h5')
print("CNN+LSTM model saved!")

# Download the model to your local machine
from google.colab import files
files.download('cnn_lstm_model.h5')

print("Model downloaded successfully!")
